# **Matrix and Vector operations/calculations**

Matrix/Vector operations are extremely important topic in high performance computing, because many of the large calculations are related to that. Also in fields like ML/ Data science, Image processing, Computer Graphics etc. Matrices are extremely important due to their properties.

In this note book first we will define a sample such problem as running example and use it to explore techniques related to Matrix/Vector operations.


Running Problem:- Diffusion of fluids (example for diffusion is dye in water).

The one dimensional partial differential equation of diffusion is as follows.

<center><image src="./img/5.jpg" width="250"/></center>

Here, `u` is the vector representing the quatities we are diffusing. For example this could represent the dye value in the location of the fluid space. In general, this will be a 2D or 3D matrix representing an actual area or volume of fluid. `D` is a value that represents the properties of the fuild we are considering. `x`, `t` represent the direction and time respectively.

Since we cant directly calculate the continuous values, we approximate above equation as follows using Eulars method.(Basically we use the differenciation limit formula for the approximation.)

<center><image src="./img/6.jpg" width="500"/></center>

Above `dt` represents the framerate(minimum time frame), `dx` represents the resolution of the image ie: smaller the dx smaller a region the matrix u cell represents. Generally we consider dx to be 1.

The basic psuedo code implementation of above for 1D (one dimension) is as follows.


<pre style="color:yellow">

# Initializing the data and conditions
u = space vector of size N
for i in range(N):
    u[i] = 0 if water else 1 #(if theres dye)

D = 1
dx = 1
t = 0
dt = 0.0001

while(True):

    print(f'time: {t}')

    u_next = vector of size N
    for i in range(N):
        u_next[i] = u[i] + D * dt * (u[(i+1)%N] + u[(i-1)%N] - 2 * u[i])


    u = u_next
    t = t + dt
    visualize(u)

</pre>

In above code all the parts which include `%N` are to solve the problem of out of bound values in the vector space. (ie: [0 - dx] or [N + dx] can be out of bound). Rather than doing that we can simply make them zero or use the same value as the closest value( same as convolutional neural networks case.)

Depending on the dimensions we need to consider, it is required to update the code. For example for 2D space diffision, we have 2 directions to consider for the diffusion gradient.

Below is an actual implementation of the diffusion algorithm for 2D space.
<center><image src="./img/7.jpg" width="400"/></center>

The below will act as the initialization of data and executor for the diffusion equation.

In [33]:
%%writefile scripts/run_experiment.py

import time

@profile
def step(grid, dt, D = 1.0):

    max_y, max_x = len(grid), len(grid[0])

    grid_next = [[0.0]*max_x]*max_y

    for i in range(max_y):
        for j in range(max_x):
            
            y_dir_gradient2 = (grid[(i + 1) % max_y][j] + grid[(i - 1) % \
                                                    max_y][j] - 2 * grid[i][j]) 
            x_dir_gradient2 = (grid[i][(j + 1) % max_x] + grid[i][(j - 1) % \
                                                     max_x] - 2 * grid[i][j])

            grid_next[i][j] = grid[i][j] + D * dt * (x_dir_gradient2 + y_dir_gradient2)

    return grid_next

def run_experiment(num_iterations, grid_shape):

    # Initial grid
    max_y, max_x = grid_shape

    grid = [[0.0]*max_x]*max_y

    # Simulating a drop of dye in the middle of our simulated region
    block_low = int(grid_shape[0] * 0.4)
    block_high = int(grid_shape[0] * 0.5)

    for i in range(block_low, block_high):
        for j in range(block_low, block_high):
            grid[i][j] = 0.005

    # Running the experiment
    st = time.time()
    for _ in range(num_iterations):
        grid = step(grid, 0.1)

    return time.time() - st

if __name__== '__main__':
    run_experiment(150, (640, 640))

Overwriting scripts/run_experiment.py


The line profling output for the above code segment is as follows.

<center><image src="./img/8.jpg" width="800"/></center>

As we can see, creation of new_grid in each step function call takes a huge amount of time per hit. This is in fact unnecessary as the dimensions of the grid does not change with time, which means we can essentially declare the list once and use it throughout all the steps. So more optimized code would be like below.

In [34]:
%%writefile scripts/run_experiment_segmented.py

import time

@profile
def step(grid, dt, grid_next, D = 1.0):

    max_y, max_x = len(grid), len(grid[0])

    for i in range(max_y):
        for j in range(max_x):
            
            y_dir_gradient2 = (grid[(i + 1) % max_y][j] + grid[(i - 1) % \
                                                    max_y][j] - 2 * grid[i][j]) 
            x_dir_gradient2 = (grid[i][(j + 1) % max_x] + grid[i][(j - 1) % \
                                                     max_x] - 2 * grid[i][j])

            grid_next[i][j] = grid[i][j] + D * dt * (x_dir_gradient2 + y_dir_gradient2)


def run_experiment(num_iterations, grid_shape):

    # Initial grid
    max_y, max_x = grid_shape

    grid = [[0.0]*max_x]*max_y

    # Simulating a drop of dye in the middle of our simulated region
    block_low = int(grid_shape[0] * 0.4)
    block_high = int(grid_shape[0] * 0.5)

    for i in range(block_low, block_high):
        for j in range(block_low, block_high):
            grid[i][j] = 0.005


    # Predefining the next grid
    grid_next = [[0.0]*max_x]*max_y

    # Running the experiment
    st = time.time()
    for _ in range(num_iterations):
        step(grid, 0.1, grid_next)

        # Swapping the variables because values inside grid does not matter for next iteration
        grid, grid_next = grid_next, grid

    return time.time() - st

if __name__== '__main__':
    run_experiment(150, (640, 640))

Overwriting scripts/run_experiment_segmented.py


<center><image src="./img/9.jpg" width="800"/></center>

We can see that the execution time has been reduced by about 20% just by separating the time consuming one time operations. But still we can see that, most of the operations we do in here is very similar. So doing this in vectorized fashion makes more sense. But python does not support vectorization natively and therefore cannot recognize whether it is possible to optimize a code segment.


In python data in lists are stores as pointers. Which means original data will be in some place in memory while the list stores it's address. This is generally great for many use cases. But not for Matrix/ Vector based situations. For example consider the `data[x][y]` list item. Since we are accesing using pointers, python need to do 2 memory lookups just to locate one item. This is generally fine. But for many such operations it can be costly. 

Also in most cases other nearby items needed for calculations as well. Therefore as an optimaization we can take that block of memory in one operation rather than locating indivudual item (caching). But if the data is fragmented in the memory this cannot be done and therefore CPU need to wait for the data to come (cache misses).

To avoid above cache missing problem, modern CPUs have few mechanisms namely `branch prediction` and `pipelining`, which try to predict the next instruction and load the relevant data before hand to improve the CPU performance. But still the most effective way of solving the mentioned problem is allocating the memory in a proper way.

In performance analysis, we can identify the how the data is getting moved to the CPU using various tools.
- Linux --> `perf` tool
- OSX   --> google's `gperftools` or provided `Instruments` tool
- Windows --> `Visual Studio Profiler `


**Important concepts to note:-**

* Difference between number of CPU instructions and CPU cycles indicate how well our code is vectorized and pipelined.
* CS (Context Switches) and migrations indicates how many times our program halted to let other applications run (may be due to I/O operations, waiting operations etc.). We cant control this happening, but can reduce the occurance by reducing blocking operations. (btw migrations means the CPU core which execute the program halt it and start the rest of the program in another core to make all cores evenly used.)
* Page faults occur when the requested memory is not available. Then kernal allocate/ read the memory to the cache. This is called `minor page fault` and problamatic in CPU bound situations. On the other hand `hard page faults` means required data is not available even in the memory, so need to read it from disk which obviously problematic in any program in general.
* It is important to lay the data in memory to reduce `cache-misses`.
* Branches in CPU optimization context is a point where the execution of the code flow change. For example if-else conditions, function calls can be considered as branches. Modern CPUs try to predict the flow before hand and load the instructions early. This can lead to more performance if prediction is correct. If they are wrong execution may take longer. This is called `branch miss`. For example some loops may run faster on sorted loops than non sorted one.


With those in mind, we can see that higher number of instructions executed for a single CPU cycle improve performance of our program. to achieve that we can make sure all the related data need for the future processing as well is pre-loaded to the cache. But since python lists store data as pointers, actual data may not in same memory block(fragmented) which can be copied to the cache. To avoid this issue, we can use python `array` module instead of lists.

Python arrays store data sequentially in the memory rather than pointers. So now kernal can load continuous memory chunks to the cache, which in turn reduce cache misses. But this does not vectorize the process we require. In order to perform vectorized CPU operations we need to use special modules which instructs python to use them.

> Due to the nature of implementation of python `array` it is slower than normal lists when creating. Not only that, it has an overhead when reading values as well. Therefore generally not good for math operations. But for storing fixed type data.

So to deal with above problems, python has a package `numpy` which stores data as contiguous chunks of memory and support vectorized operations. Below examples show the improvements of numpy compared to normal lists on vectorizable operations.

Consider taking a sum of square of list values.

In [35]:
from array import array
import numpy as np

In [36]:
def sum_square_lists(vector):
    sum_val = 0
    for i in vector:
        sum_val += i
    return sum_val

def sum_square_list_comprehension(vector):

    return sum([i*i for i in vector])

def sum_square_array(vector):
    sum_val = 0
    for i in vector:
        sum_val += i
    return sum_val

def sum_square_numpy(vector):
    return np.sum(vector*vector)

def sum_square_np_vectorized(vector):
    return np.dot(vector, vector)


In [37]:
vector = [i for i in range(1_000_000)]

In [38]:
%%timeit
sum_square_lists(vector)

29.8 ms ± 995 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [39]:
%%timeit
sum_square_list_comprehension(vector)

71.5 ms ± 1.53 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [40]:
vector = array('l', range(1_000_000))

In [41]:
%%timeit
sum_square_array(vector)

35.6 ms ± 910 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [42]:
vector = np.arange(1_000_000)

In [43]:
%%timeit
sum_square_numpy(vector)

1.8 ms ± 8.31 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [44]:
%%timeit
sum_square_np_vectorized(vector)

439 µs ± 2.26 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


As we can clearly see numpy based both operations have significant speedups compared to the generic python data stuctures. In the numpy based loop we imply the numpy that each element should multiply by itself, which in turn gives us significant performance.


Additionally the vectorized function performs way faster than its equivalent normal function.

With those in mind we can use numpy for our diffusion problem.

In [62]:
%%writefile scripts/run_experiment_vectorized.py

import time
from numpy import roll, zeros


def shift_vals(grid):

    return (roll(grid, +1, 0) +  
            roll(grid, -1, 0) +
            roll(grid, +1, 1) +
            roll(grid, -1, 1) -
            4 * grid )


@profile
def step(grid, dt, D = 1.0):
    return grid + dt * D * shift_vals(grid)


def run_experiment(num_iterations, grid_shape):

    # Initial grid
    max_y, max_x = grid_shape

    grid = zeros((max_y, max_x))

    # Simulating a drop of dye in the middle of our simulated region
    block_low = int(grid_shape[0] * 0.4)
    block_high = int(grid_shape[0] * 0.5)

    grid[block_low:block_high, block_low:block_high] = 0.005

    # Running the experiment
    st = time.time()
    for _ in range(num_iterations):
        grid = step(grid, 0.1)

    return time.time() - st

if __name__== '__main__':
    run_experiment(150, (640,640))

Overwriting scripts/run_experiment_vectorized.py


Obviously this has less code lines than the previous implementation. So line profiler is not useful. But still we can see the overall performance as below.

<center><image src="./img/10.jpg" width="800"/></center>

The total time for 150 iterations is less than 2 seconds!

Above happens for many reasons. Mainly numpy arrays are specialized and does not support all the python list functionalities. They are expecting a single data type (in our case integer) unlike python list which can accept any value. Also as mentioned they keep values not references and store them in contigues manner. And also supports CPU level vectorization instructions. All those combined provide massive performance in applicable scenarios.

But above can be further optimized. In previous implementation, in each step new array is getting allocated to store the data after rolling. This is time consuming task in a CPU bound work. Therefore we can reduce that time by first defining a memory space and reusing it for further calculations (in this case it is possible). To do that we need to understand how numpy memory allocation happens in certain scenarios.

In [63]:
array1 = np.random.random((10,10))
array2 = np.random.random((10,10))
print(id(array1))
print(id(array2))

2412461656176
2412388653552


In [64]:
array1 = array1 + array2
print(id(array1))

2412388654128


In [65]:
array1 = np.random.random((10,10))
array2 = np.random.random((10,10))
print(id(array1))
print(id(array2))

2412393274608
2412388654128


In [66]:
array1 += array2
print(id(array1))

2412393274608


As we can see in the first method, after the addition operation, array1 id has changed which indicate the memory address change.

But on the other hand, in second addition method array1 id does not change. Which means it does the addition in place. Therefore by using this method, we can reduce memory allocations of our programs.

In [67]:
%%timeit array1, array2 = np.random.random((2, 100, 100))
array1 = array1 + array2

3.89 µs ± 65.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [68]:
%%timeit array1, array2 = np.random.random((2, 100, 100))
array1 += array2

2.82 µs ± 39.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [85]:
%%writefile scripts/run_experiment_mem_optimized.py

import time
from numpy import roll, zeros, copyto


def shift_vals(grid, next_grid):
    copyto(next_grid, grid)
    next_grid +=  roll(grid, +1, 0)
    next_grid +=  roll(grid, -1, 0)
    next_grid +=  roll(grid, +1, 1)
    next_grid +=  roll(grid, -1, 1)
    next_grid *= -4

@profile
def step(grid, next_grid, dt, D = 1.0):
    shift_vals(grid, next_grid)
    next_grid *= dt * D
    grid += next_grid


def run_experiment(num_iterations, grid_shape):

    # Initial grid
    max_y, max_x = grid_shape
    grid = zeros((max_y, max_x))
    next_grid = zeros((max_y, max_x))

    # Simulating a drop of dye in the middle of our simulated region
    block_low = int(grid_shape[0] * 0.4)
    block_high = int(grid_shape[0] * 0.5)

    grid[block_low:block_high, block_low:block_high] = 0.005

    # Running the experiment
    st = time.time()
    for _ in range(num_iterations):
        step(grid, next_grid, 0.1)
        grid, next_grid = next_grid, grid # Inplace variable swap

    return time.time() - st

if __name__== '__main__':
    run_experiment(150, (640,640))

Overwriting scripts/run_experiment_mem_optimized.py


> It is important to remember that since we want each operation to be in-place, whenever we do a vector operation, we must put it on its own line. This can make something as simple as A = A * B + C become quite convoluted.

The time analysis of above script is as follows.

<center><image src="./img/11.jpg" width="700"/></center>

If we inspect the time taken by the above memory optimized version, it performs considerably better than the just vectorized version. This is mainly due to less memory allocation/change during the execution.

Also we can see that majority of the time consumed by our program goes to shift_vals() function. This is due to memory operations of numpy roll function. Therefore we can optimize that by using a more specialized version for our usecase. But this comes with additional programming complexity and readability problems. So it is important to do such things carefully validating the pros and cons.

In [88]:
%%writefile scripts/run_experiment_further_mem_optimized.py

import time
from numpy import roll, zeros, copyto, asarray, all

def shift_sum_optimized(grid, shift_val, axis, next_grid):
    '''
    This is faster because this does the addition along with the numpy fast indexing operations.
    Therefore we dont need to do the addition operations separately.
    
    '''
    if(axis==0):
        next_grid[shift_val:,:] += grid[:-shift_val,:]
        next_grid[:shift_val,:] += grid[-shift_val:,:]
    elif(axis==1):
        next_grid[:,shift_val:] += grid[:,:-shift_val]
        next_grid[:,:shift_val] += grid[:,-shift_val:]

def test_shift_sum_optimized():
    rollee = asarray([[1, 2], [3, 4]])
    for shift in (+1, -1):
        for axis in (0, 1):
            out = asarray([[6, 3], [9, 2]])

            expected_result = roll(rollee, shift, axis=axis) + out
            shift_sum_optimized(rollee, shift, axis, out)

            assert all(expected_result == out)

def shift_vals(grid, next_grid):
    copyto(next_grid, grid)
    shift_sum_optimized(grid, +1, 0, next_grid)
    shift_sum_optimized(grid, -1, 0, next_grid)
    shift_sum_optimized(grid, +1, 1, next_grid)
    shift_sum_optimized(grid, -1, 1, next_grid)
    next_grid *= -4

@profile
def step(grid, next_grid, dt, D = 1.0):
    shift_vals(grid, next_grid)
    next_grid *= dt * D
    grid += next_grid


def run_experiment(num_iterations, grid_shape):

    # Initial grid
    max_y, max_x = grid_shape
    grid = zeros((max_y, max_x))
    next_grid = zeros((max_y, max_x))

    # Simulating a drop of dye in the middle of our simulated region
    block_low = int(grid_shape[0] * 0.4)
    block_high = int(grid_shape[0] * 0.5)

    grid[block_low:block_high, block_low:block_high] = 0.005

    # Running the experiment
    st = time.time()
    for _ in range(num_iterations):
        step(grid, next_grid, 0.1)
        grid, next_grid = next_grid, grid # Inplace variable swap

    return time.time() - st
            
test_shift_sum_optimized()

if __name__== '__main__':
    run_experiment(150, (640,640))

Overwriting scripts/run_experiment_further_mem_optimized.py


Time consumption is as follows for the above further improved version.

<center><image src="./img/12.jpg" width="700"/></center>

We can see that execution time has been reduced further compared to the previous implementation.

One annoyance we encounter when doing in-place operations using numpy is that we need to separate the operations explicitly or they will create temporary variables in the middle (eg:- A * B + C). 

To help in such situations there are many modules. `numexpr` is one such module which can take a vector expression and compile it to a efficient code optimized for low cache misses and low temporary space usages. Sample usage of numexpr package is shown below.

In [92]:
%%writefile scripts/run_experiment_numexpr_optimized.py

import time
from numpy import roll, zeros, copyto, asarray, all
from numexpr import evaluate

def shift_sum_optimized(grid, shift_val, axis, next_grid):
    '''
    This is faster because this does the addition along with the numpy fast indexing operations.
    Therefore we dont need to do the addition operations separately.
    
    '''
    if(axis==0):
        next_grid[shift_val:,:] += grid[:-shift_val,:]
        next_grid[:shift_val,:] += grid[-shift_val:,:]
    elif(axis==1):
        next_grid[:,shift_val:] += grid[:,:-shift_val]
        next_grid[:,:shift_val] += grid[:,-shift_val:]

def test_shift_sum_optimized():
    rollee = asarray([[1, 2], [3, 4]])
    for shift in (+1, -1):
        for axis in (0, 1):
            out = asarray([[6, 3], [9, 2]])

            expected_result = roll(rollee, shift, axis=axis) + out
            shift_sum_optimized(rollee, shift, axis, out)

            assert all(expected_result == out)

def shift_vals(grid, next_grid):
    copyto(next_grid, grid)
    shift_sum_optimized(grid, +1, 0, next_grid)
    shift_sum_optimized(grid, -1, 0, next_grid)
    shift_sum_optimized(grid, +1, 1, next_grid)
    shift_sum_optimized(grid, -1, 1, next_grid)
    next_grid *= -4

@profile
def step(grid, next_grid, dt, D = 1.0):
    shift_vals(grid, next_grid)
    evaluate("next_grid * D * dt + grid", out=next_grid)


def run_experiment(num_iterations, grid_shape):

    # Initial grid
    max_y, max_x = grid_shape
    grid = zeros((max_y, max_x))
    next_grid = zeros((max_y, max_x))

    # Simulating a drop of dye in the middle of our simulated region
    block_low = int(grid_shape[0] * 0.4)
    block_high = int(grid_shape[0] * 0.5)

    grid[block_low:block_high, block_low:block_high] = 0.005

    # Running the experiment
    st = time.time()
    for _ in range(num_iterations):
        step(grid, next_grid, 0.1)
        grid, next_grid = next_grid, grid # Inplace variable swap

    return time.time() - st
            
test_shift_sum_optimized()

if __name__== '__main__':
    run_experiment(150, (640,640))

Writing scripts/run_experiment_numexpr_optimized.py


Important thing to note in using this module is that, it is optimized for caching. Therefore if the program is small enough to fit in the cache then there is no point using this. Also compiling the vector operations specifically add a considerable overhead to the program. To get a noticable performance improvement we need to have large enough program complexity.


Other than numpy, `pandas` library is also very popular in python world for data manipulation tasks. Its table like data structure (data frame) performs operations eagerly to the data columns and often generate intermediate arrays while doing the operation. Therefore it is generally recommended to expect three to five times memory usage than the actual data while manipulating dataframes. Typically pandas works well for datasets up to 10GB assuming the computer have enough memory. Following are few tips that we might want to focus in pandas.

1. Pandas uses numpy data types mainly. But for additional features it has its own data types. Depending on the situation we might want to choose correct ones. (for example if our integer dataset contains NaN values data type would be `Int64` rather than `int64` (note the capitalization of data type name))
2. Try to use `apply` function rather than iterrows or iloc methods to improve performance.
3. Pandas `concat` function creates entirely new dataframe/ series upon its use. Therefore should avoid repeated calls as much as possible.
4. Install the optional dependencies `numexpr` and `bottleneck` for additional performance improvements.
5. For categorical type data, use `categorical` datatype in pandas for improved speed in calculations.
